## An agent to validate radiology report based on ACR guidelines
The American College of Radiology (ACR), as the leading professional organization for radiologists, has established comprehensive guidelines for standardized radiology reporting to ensure diagnostic accuracy and patient care quality. In many healthcare settings, the current workflow requires senior radiologists to review and validate reports generated by junior colleagues—a critical but time-intensive process that creates bottlenecks in medical imaging departments already struggling with high workloads. To streamline this process while maintaining quality standards, Amazon NOVA's advanced multimodal AI capabilities can be leveraged to automatically evaluate radiology reports against ACR's established criteria, providing real-time feedback and guidance. This AI-assisted solution can instantly identify missing elements, suggest improvements based on ACR guidelines, and help junior radiologists generate more accurate and complete reports, ultimately reducing the review burden on senior radiologists while maintaining high reporting standards.

### Prepare the data
The ACR validation documents are in the folder ACRdocs. Upload the same in an S3 bucket. Change the name of the S3 bucket in the lambda function on line 38 in `lambda_handler`

In [55]:
!python3 -m pip install --upgrade -q botocore
!python3 -m pip install --upgrade -q boto3
!python3 -m pip install --upgrade -q awscli

### Configure `AWS_PROFILE`
Configure the `AWS_PROFILE` using the usual process as described [here](https://docs.aws.amazon.com/cli/v1/userguide/cli-configure-files.html)

In [75]:
import os
os.environ['AWS_PROFILE'] = 'rad-agent-demo'

In [76]:
import boto3
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging
print(boto3.__version__)

1.37.33


#### Configure the `boto3` clients

In [77]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

In [78]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-west-2', '048051882663')

### Write the agent instruction

In [79]:
# configuration variables
suffix = f"{region}-{account_id}"
stack_name = "RadiologyReportValidator"
agent_name = f"{stack_name}-Agent"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}-{stack_name}"
agent_role_name = f'AmazonBRkExRoleForAgents_{agent_name}'
agent_foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"
agent_description = "Agent for Validating Radiology Reports"
agent_instruction = "You are a Radiology Report Validator, helping junior radiologist \
                    write reports in adherence to the ACR guidance criterion. Does the radiology report adheres to the ACR guidelines mentioned in the document? \
                    Is it detailed enough to provide a diagnosis? \
                    Is the report missing any key anatomical structures? \
                    Does the report meet the \
                    quality standards of the ACR guidelines? Please provide a terse actionable feedback and do not try to summarize the report itself. ?"
agent_action_group_name = "RadiologyActionGroup"
agent_action_group_description = "Actions for Validating Radiology Reports or from Machine Learning Models"
agent_alias_name = f"{agent_name}-alias"


### Assign the roles

In [ ]:
lambda_function_role = f"{stack_name}-LambdaExecutionRole"
lambda_function_name = f"{stack_name}-{account_id}-Lambda"
print(lambda_function_name)

# attach the s3 full access policy to the Lambda function role
s3_full_access_policy_arn = "arn:aws:iam::aws:policy/AmazonS3FullAccess"
bedrock_policy_arn = "arn:aws:iam::aws:policy/AmazonBedrockFullAccess"
AWSLambdaBasicExecutionRole_arn = "arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole"

# create role for the Lambda function
try:
    iam_client.create_role(
        RoleName=lambda_function_role,
        AssumeRolePolicyDocument=json.dumps({
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Principal": {
                        "Service": "lambda.amazonaws.com"
                    },
                    "Action": "sts:AssumeRole"
                }
            ]
        }),
        Description="Role for Lambda function to access S3 and other resources"
    )

    iam_client.attach_role_policy(
        RoleName=lambda_function_role,
        PolicyArn=s3_full_access_policy_arn
        
    )

except Exception as e:
    print("Error creating role: ", e)
    print("Role already exists. Continuing...")
    



RadiologyReportValidator-048051882663-Lambda
Error creating role:  An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name RadiologyReportValidator-LambdaExecutionRole already exists.
Role already exists. Continuing...


In [81]:
Role=iam_client.get_role(RoleName=lambda_function_role)['Role']['Arn']
print(lambda_function_role)

RadiologyReportValidator-LambdaExecutionRole


In [82]:
iam_client.attach_role_policy(
        RoleName=lambda_function_role,
        PolicyArn=AWSLambdaBasicExecutionRole_arn
    )
iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn=bedrock_policy_arn
)
# assign the created role to a lambda function
lambda_client.update_function_configuration(
    FunctionName=lambda_function_name,
    Role=iam_client.get_role(RoleName=lambda_function_role)['Role']['Arn'],
    Timeout=900
)

{'ResponseMetadata': {'RequestId': '24b91a41-eb91-423e-9988-a12f42961c33',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 13 Apr 2025 21:46:59 GMT',
   'content-type': 'application/json',
   'content-length': '1457',
   'connection': 'keep-alive',
   'x-amzn-requestid': '24b91a41-eb91-423e-9988-a12f42961c33'},
  'RetryAttempts': 0},
 'FunctionName': 'RadiologyReportValidator-048051882663-Lambda',
 'FunctionArn': 'arn:aws:lambda:us-west-2:048051882663:function:RadiologyReportValidator-048051882663-Lambda',
 'Runtime': 'python3.12',
 'Role': 'arn:aws:iam::048051882663:role/RadiologyReportValidator-LambdaExecutionRole',
 'Handler': 'lambda_function.lambda_handler',
 'CodeSize': 6313,
 'Description': '',
 'Timeout': 900,
 'MemorySize': 128,
 'LastModified': '2025-04-13T21:46:59.000+0000',
 'CodeSha256': '1/EaWlndwdxynJVU8mZrwBD2nF1qVRDI4jmB3zXAK0M=',
 'Version': '$LATEST',
 'TracingConfig': {'Mode': 'PassThrough'},
 'RevisionId': '81a7c42a-dd53-46e2-ae5a-14325fd3a629',
 'State': 

#### Write and Upload the lambda function

In [83]:
# Upload the lambda function
# Package up the lambda function code and deploy to Lambda function
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("lambda/lambda_function.py", arcname="lambda_function.py")
z.close()
zip_content = s.getvalue()

lambda_function = lambda_client.update_function_code(
    FunctionName=lambda_function_name,
    ZipFile=zip_content,
    )
lambda_function_arn = lambda_function['FunctionArn']


### Create IAM policies for agent


In [84]:
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)
try:
    agent_bedrock_policy = iam_client.create_policy(
        PolicyName=agent_bedrock_allow_policy_name,
        PolicyDocument=bedrock_policy_json
    )
except Exception:
    print(f"Policy {agent_bedrock_allow_policy_name} already exists")

Policy RadiologyReportValidator-Agent-ba-us-west-2-048051882663-RadiologyReportValidator already exists


In [ ]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
try:
    agent_role = iam_client.create_role(
        RoleName=agent_role_name,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )


    # Pause to make sure role is created
    time.sleep(10)
        
except Exception as e:
    agent_role = iam_client.get_role(RoleName=agent_role_name)
    print(f"Error creating role: {e}. Make sure it doesnt exist already")

iam_client.attach_role_policy(
        RoleName=agent_role_name,
        PolicyArn=agent_bedrock_policy['Policy']['Arn']
    )

Error creating role: An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name AmazonBRkExRoleForAgents_RadiologyReportValidator-Agent already exists.. Make sure it doesnt exist already


In [32]:
print(agent_role)

{'Role': {'Path': '/', 'RoleName': 'AmazonBRkExRoleForAgents_RadiologyReportValidator-Agent', 'RoleId': 'AROAQWMA5RKTRUUO5CRIF', 'Arn': 'arn:aws:iam::048051882663:role/AmazonBRkExRoleForAgents_RadiologyReportValidator-Agent', 'CreateDate': datetime.datetime(2025, 4, 13, 19, 23, 47, tzinfo=tzutc()), 'AssumeRolePolicyDocument': {'Version': '2012-10-17', 'Statement': [{'Effect': 'Allow', 'Principal': {'Service': 'bedrock.amazonaws.com'}, 'Action': 'sts:AssumeRole'}]}, 'MaxSessionDuration': 3600, 'RoleLastUsed': {'LastUsedDate': datetime.datetime(2025, 4, 13, 19, 44, 15, tzinfo=tzutc()), 'Region': 'us-east-2'}}, 'ResponseMetadata': {'RequestId': 'ef7034b8-abf8-461b-bdc6-2ba84ccc9c83', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 13 Apr 2025 21:09:45 GMT', 'x-amzn-requestid': 'ef7034b8-abf8-461b-bdc6-2ba84ccc9c83', 'content-type': 'text/xml', 'content-length': '1014'}, 'RetryAttempts': 0}}


### Create Agent

In [67]:
try:
    response = bedrock_agent_client.create_agent(
        agentName=agent_name,
        agentResourceRoleArn=agent_role['Role']['Arn'],
        description=agent_description,
        idleSessionTTLInSeconds=1800,
        foundationModel=agent_foundation_model,
        instruction=agent_instruction,
    )
    agent_id = response['agent']['agentId']
    print(response)

except:
    print("Agent already exists, skipping creation")
    agent_info = [agent for agent in bedrock_agent_client.list_agents()['agentSummaries'] if agent['agentName']==agent_name][0]
    print(agent_info)
    agent_id = agent_info['agentId']
    agent_version = bedrock_agent_client.list_agent_versions(agentId=agent_id)['agentVersionSummaries'][0]['agentVersion']
    response = bedrock_agent_client.list_agent_action_groups(
        agentId=agent_id,
        agentVersion=agent_version
    )
    print(response['actionGroupSummaries'][0])
    action_group_id = response['actionGroupSummaries'][0]['actionGroupId']
    print(f"Using agent_id {agent_id} and action_group_id {action_group_id}")

{'ResponseMetadata': {'RequestId': 'f6df756b-1237-4b66-bb51-6b5bb4a541cf', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Sun, 13 Apr 2025 21:25:46 GMT', 'content-type': 'application/json', 'content-length': '1162', 'connection': 'keep-alive', 'x-amzn-requestid': 'f6df756b-1237-4b66-bb51-6b5bb4a541cf', 'x-amz-apigw-id': 'I-vKGHMcPHcEUhA=', 'x-amzn-trace-id': 'Root=1-67fc2bd9-432181f71fc191af05908de7'}, 'RetryAttempts': 0}, 'agent': {'agentArn': 'arn:aws:bedrock:us-west-2:048051882663:agent/L5JXYO5NBP', 'agentCollaboration': 'DISABLED', 'agentId': 'L5JXYO5NBP', 'agentName': 'RadiologyReportValidator-Agent', 'agentResourceRoleArn': 'arn:aws:iam::048051882663:role/AmazonBRkExRoleForAgents_RadiologyReportValidator-Agent', 'agentStatus': 'CREATING', 'createdAt': datetime.datetime(2025, 4, 13, 21, 25, 45, 965158, tzinfo=tzutc()), 'description': 'Agent for Validating Radiology Reports', 'foundationModel': 'anthropic.claude-3-sonnet-20240229-v1:0', 'idleSessionTTLInSeconds': 1800, 'instructio

## Write Agent functions
Here we create agent functions which we would like to call as the agent processes the input radiology report. A typical agent_function contains the following fields:
- `name`: The name of the function
- `description`: Description of the function
- `parameters`: A dictionary of parameters to be passed in the function 
The `parameter` field contains the name of the parameter, its description, type as well as whether it is a required variable 

In [68]:
agent_functions = [
    {
        'name': 'run_validator',
        'description': 'validates the radiology report for a given report',
        'parameters': {
            "report": {
                "description": "The radiology report for a given patient",
                "required": True,
                "type": "string"
            }
        }
    }, 
    {
        'name' : 'check_radiology_report',
        'description': 'Check of the given text contains a radiology report', 
        'parameters': {
            'report': {
                'description': 'If the text is not a radiology report. Ask for the user to \
                provide the radiology report.',
                'required': True,
                'type': 'string'
            }
        }
    },
    {
        'name': 'download_guidance_document',
        'description': 'Download the ACR guidance document from the S3 bucket. If the document is not \
        available, ask the user to provide the document.',
        'parameters': {
            'report': {
                'description': 'Name of the modality or the anatomical structure to download the ACR guidance document',
                'required': True,
                'type': 'string'
            }
        }
    },
    {
        'name': 'identify_anatomical_structures',
        'description': 'Identify the anatomical structures in the given text. Identify which of the following anatomical structures \
            are present in the text: \
                - Brain \
                - Spine \
                - Chest \
                - Abdomen \
                - Pelvis \
                - Extremities\
                - Transthoracic \
                and return the value from the list.',
        'parameters': {
            'report': {
                'description': 'The radiology report for a given patient',
                'required': True,
                'type': 'string'
            }
        }
    },
    
]

In [69]:
try:
    agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function_arn
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)
except:
    print("Action group already exists")
    agent_action_group_response = bedrock_agent_client.update_agent_action_group(
        agentId=agent_id,
        actionGroupId=action_group_id,
        agentVersion='DRAFT',
        actionGroupExecutor={
            'lambda': lambda_function_arn
        },
        actionGroupName=agent_action_group_name,
        functionSchema={
            'functions': agent_functions
        },
        description=agent_action_group_description
    )
    print(agent_action_group_response)

### Permissions for the lambda function
In this lamda function, required permissions are given. Here we are using bedrock in the agent, so appropriate permissions are given.

In [52]:
try:
    response = lambda_client.add_permission(
        FunctionName=lambda_function_name,
        StatementId='allow_bedrock',
        Action='lambda:InvokeFunction',
        Principal='bedrock.amazonaws.com',
        SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
    )
    print(response)
except: 
    print("Permission already exists")

{'ResponseMetadata': {'RequestId': '184677f0-f1d6-4a64-8524-d4dd79d1895c', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Sun, 13 Apr 2025 21:19:56 GMT', 'content-type': 'application/json', 'content-length': '372', 'connection': 'keep-alive', 'x-amzn-requestid': '184677f0-f1d6-4a64-8524-d4dd79d1895c'}, 'RetryAttempts': 0}, 'Statement': '{"Sid":"allow_bedrock","Effect":"Allow","Principal":{"Service":"bedrock.amazonaws.com"},"Action":"lambda:InvokeFunction","Resource":"arn:aws:lambda:us-west-2:048051882663:function:RadiologyReportValidator-048051882663-Lambda","Condition":{"ArnLike":{"AWS:SourceArn":"arn:aws:bedrock:us-west-2:048051882663:agent/5DNO9EAAJI"}}}'}


#### Prepare the agent

In [70]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

{'ResponseMetadata': {'RequestId': 'd1d17119-0a69-426e-b9b0-5a1d414fab9e', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Sun, 13 Apr 2025 21:25:58 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': 'd1d17119-0a69-426e-b9b0-5a1d414fab9e', 'x-amz-apigw-id': 'I-vMEG74PHcEqGg=', 'x-amzn-trace-id': 'Root=1-67fc2be6-02670c4a554fe0ab6645b372'}, 'RetryAttempts': 0}, 'agentId': 'L5JXYO5NBP', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2025, 4, 13, 21, 25, 58, 754145, tzinfo=tzutc())}


## Test case

In [73]:

test_query ="Here is the radiology report: FINAL REPORT EXAMINATION : CHEST ( PORTABLE AP ) INDICATION : History :___ F with ett placement TECHNIQUE : Upright AP view of the chest COMPARISON : None . Patient is currently listed as EU critical . \
FINDINGS : Endotracheal tube tip terminates approximately a 4.6 cm from the carina . Enteric tube tip terminates within the distal esophagus and should be advanced by at least 11 cm .Heart size is normal . Gaseous distension of the stomach is noted "
test_query = "x"
session_id:str = str(uuid.uuid1())
print(agent_id)

response = bedrock_agent_runtime_client.invoke_agent(
      inputText=test_query,
      agentId=agent_id,
      agentAliasId="TSTALIASID", 
      sessionId=session_id,
      enableTrace=True, 
      endSession=False,
      sessionState={}
)
print(response['completion'])
answer=""
for event in response["completion"]:
    print(event.keys())
    print(event)
    if 'chunk' in event:
        chunk_obj = event['chunk']
        if 'bytes' in chunk_obj:
            # Decode the bytes and append to the answer
            chunk_data = chunk_obj['bytes'].decode('utf-8')
            answer += chunk_data

L5JXYO5NBP
dict_keys(['trace'])
{'trace': {'agentAliasId': 'TSTALIASID', 'agentId': 'L5JXYO5NBP', 'agentVersion': 'DRAFT', 'callerChain': [{'agentAliasArn': 'arn:aws:bedrock:us-west-2:048051882663:agent-alias/L5JXYO5NBP/TSTALIASID'}], 'eventTime': datetime.datetime(2025, 4, 13, 21, 28, 31, 955086, tzinfo=tzutc()), 'sessionId': '3f483a4e-18ae-11f0-bd12-f204dc4090c7', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'inferenceConfiguration': {'maximumLength': 2048, 'stopSequences': ['</function_calls>', '</answer>', '</error>'], 'temperature': 0.0, 'topK': 250, 'topP': 1.0}, 'text': '{"system":" You are a Radiology Report Validator, helping junior radiologist                     write reports in adherence to the ACR guidance criterion. Does the radiology report adheres to the ACR guidelines mentioned in the document?                     Is it detailed enough to provide a diagnosis?                     Is the report missing any key anatomical structures?                     Does 

EventStreamError: An error occurred (accessDeniedException) when calling the InvokeAgent operation: Access denied when calling Bedrock. Check your request permissions and retry the request.

In [28]:
# Print the final answer
print(answer)

The provided chest radiograph report does not fully adhere to the ACR guidelines. It is missing key components like:

- Patient positioning and degree of inspiration
- Systematic evaluation of lung fields, bony structures, mediastinum, and pleural spaces  
- Mention of any consolidations, effusions, or masses
- An overall impression summarizing the significant findings

To improve the report and meet the ACR standards, the radiologist should include these missing elements in a more comprehensive and structured format.
